In [1]:
import ipywidgets as wg
from ipywidgets import HBox, VBox, Layout
import pandas as pd

In [2]:
tax_tables = pd.read_excel('TaxTables.xlsx', usecols = 'A:L')

In [3]:
### FIRST VBOX

## Sliders
style = {'description_width': '135px'}
# Tax
tax_year_slider = wg.IntSlider(style=style, description='Tax Year',value=2019, min=2017, max=2019)
pay_freq_slider = wg.SelectionSlider(style=style, description='Pay Freq', value=12, options=[('Annually',1), ('Semiannually',2), ('Quarterly',4), ('Monthly',12), ('Semimonthly',24), ('Biweekly',26), ('Weekly',52)])
filing_slider = wg.IntSlider(style=style, description='Filing',value=1, min=1, max=2)

## Float Text Boxes
style = {'description_width': '135px'}
layout=Layout(width='220px')
# Tax
bracket_min = wg.FloatText(value=0, style=style, layout=layout, description='Bracket Min:', disabled=True)
wage_amt = wg.FloatText(value=0, style=style, layout=layout, description='Wage Amount:', disabled=True)
bracket_max = wg.FloatText(value=0, style=style, layout=layout, description='Bracket Max:', disabled=True)
allowance_rt = wg.FloatText(value=0, style=style, layout=layout, description='Allowance Rate:', disabled=True)
bracket_con = wg.FloatText(value=0, style=style, layout=layout, description='Bracket Constant:', disabled=True)
bracket_pr = wg.FloatText(value=0, style=style, layout=layout, description='Bracket Percent:', disabled=True)
# Analysis
extra_total = wg.FloatText(value=0, style=style, layout=layout, description='Target - Net:', disabled=True)
extra_pr = wg.FloatText(value=0, style=style, layout=layout, description='Target / Net:', disabled=True, readout_format='.0%')



### SECOND VBOX

## Float Text Boxes
style = {'description_width': '150px'}
layout=Layout(width='250px')
# Budget
gross_target = wg.FloatText(value=0, style=style, layout=layout, description='Gross Target:', disabled=False)
admin_fee = wg.FloatText(value=0, style=style, layout=layout, description='Admin Fee:', disabled=True)
attrn_fee = wg.FloatText(value=0, style=style, layout=layout, description='Attrition Fee:', disabled=True)
er_taxes = wg.FloatText(value=0, style=style, layout=layout, description='Employer Taxes:', disabled=True)
taxable_gross_housing = wg.FloatText(value=0, style=style, layout=layout, description='Taxable Gross + Housing:', disabled=True)
taxable_gross = wg.FloatText(value=0, style=style, layout=layout, description='Taxable Gross:', disabled=True)
allowances = wg.FloatText(value=0, style=style, layout=layout, description='Total Allowances:', disabled=True)
fit = wg.FloatText(value=0, style=style, layout=layout, description='Federal Income Taxes:', disabled=True)
eess_amt = wg.FloatText(value=0, style=style, layout=layout, description='Employee SS:', disabled=True)
eemd_amt = wg.FloatText(value=0, style=style, layout=layout, description='Employee MD:', disabled=True)
net_pay = wg.FloatText(value=0, style=style, layout=layout, description='Net Pay:', disabled=True)
housing = wg.FloatText(value=0, style=style, layout=layout, description='Housing:', disabled=False)
total_net = wg.FloatText(value=0, style=style, layout=layout, description='Total Net:', disabled=False)

## Sliders
style = {'description_width': '0px'}
layout=Layout(width='175px')
layout2=Layout(width='250px')
# Budget
gross_target_slider = wg.FloatSlider(style=style, layout=layout, description='Gross Target', value=0, min=0, max=10000, step=1, readout=False)
admin_fee_slider = wg.FloatSlider(style=style, layout=layout2, description='Admin Fee', value=0.10, min=0.00, max=0.2, step=0.01, readout_format='.0%')
attrn_fee_slider = wg.FloatSlider(style=style, layout=layout2, description='Attrition Fee', value=0.05, min=0.00, max=0.2, step=0.01, readout_format='.0%')
allowances_slider = wg.IntSlider(style=style, layout=layout2, description='Allowances', value=0, min=0, max=10, step=1)
housing_slider = wg.FloatSlider(style=style, layout=layout, description='Housing',value=0,min=0, max=5000, step=1, readout=False)
total_net_slider = wg.FloatSlider(style=style, layout=layout, description='Total Net', value=0, min=0, max=10000, step=1, readout=False)


#1 For Displaying in a different Cell
output = wg.Output()

def get_tax_table(*args):
    """
    Function that selects the relevant tax table 
    based off year, pay frequency, and filing.
    """
    
    tax_yr = tax_year_slider.value
    pay_freq = pay_freq_slider.value
    filing = filing_slider.value

    # Filters that will find focused tax table
    filter_1 = tax_tables.loc[:, 'Year']==tax_yr
    filter_2 = tax_tables.loc[:, 'PayFrequency']==pay_freq
    filter_3 = tax_tables.loc[:, 'Filing']==filing

    # Get tax_tables filtered
    tax_tables_filtered = tax_tables[filter_1 & filter_2 & filter_3]
    
    # Get top row values to use in new row
    top_row = tax_tables_filtered.iloc[0]
    erss = top_row.loc['erss']
    ermd = top_row.loc['ermd']
    eess = top_row.loc['eess']
    eemd = top_row.loc['eemd']
    b_al = top_row.loc['Allowance']
    b_min = top_row.loc['Min']
    b_con = top_row.loc['Constant']

    # Add row to top
    new_vals = [tax_yr, filing, pay_freq, b_al, erss, ermd, eess, eemd, 0, b_min, 0, 0]
    new_row = pd.Series(new_vals, index=tax_tables_filtered.columns)
    tax_tables_filtered = tax_tables_filtered.append(new_row, ignore_index=True).sort_values(['Min'], ascending=True)
    tax_tables_filtered.reset_index(drop=True, inplace=True)
    
    return tax_tables_filtered


def get_tax_table_bracket_row(tax_table, wage_amount_values):
    """
    Function that selects the relevant row in the tax table 
    based off Wage Amount found inbetween min and max.
    """

    # Add Wage Amount values as new column
    tax_table.loc[:, 'WageAmount'] = wage_amount_values

    # Replace Negative Wages for 0
    tax_table = tax_table.clip(lower=0)
    
    # Find row where Wage Amount is inbetween Min and Max. This will be your bracket row
    f_min = tax_table['WageAmount'] >= tax_table['Min']
    f_max = tax_table['WageAmount'] < tax_table['Max']
    
    # Return bracket row where wage amount if in between min and max.
    return tax_table[f_min & f_max] #bracket_row


    
def run_calc_net(tax_t):
    """
    Function that runs calculations and adjusments
    when Target Net is fixed.
    """
    
    # Get current variables
    tn = total_net_slider.value
    hs = housing_slider.value
    al = allowances_slider.value
    atr = attrn_fee_slider.value
    adf = admin_fee_slider.value

    # Get columns needed for WA calculation
    eess = tax_t.loc[:, 'eess']
    eemd = tax_t.loc[:, 'eemd']
    b_pr = tax_t.loc[:, 'Percent']
    b_al = tax_t.loc[:, 'Allowance']
    b_min = tax_t.loc[:, 'Min']
    b_con = tax_t.loc[:, 'Constant']

    # f9 =(tn+b_con-hs*(1-eess-eemd)-b_pr*(al*b_al+b_min))/(1-eess-eemd-b_pr)-al*b_al
    # Broken Down
    f1 = tn + b_con
    f2 = hs * (1 - eess - eemd)
    f3 = b_pr * (al * b_al + b_min)
    f4 = 1 - eess - eemd - b_pr
    f5 = al * b_al
    
    # Get Wage amount column
    wage_amt_column = ((f1 - f2 - f3) / f4) - f5
    
    # Find row in Tax table where wage amount is inbetween min and max
    bracket_row = get_tax_table_bracket_row(tax_t, wage_amt_column)
    
    # Use bracket row to find remaining Amounts
    erss = bracket_row.loc[:, 'erss']
    ermd = bracket_row.loc[:, 'ermd']
    eess = bracket_row.loc[:, 'eess']
    eemd = bracket_row.loc[:, 'eemd']
    b_pr = bracket_row.loc[:, 'Percent']
    b_al = bracket_row.loc[:, 'Allowance']
    b_min = bracket_row.loc[:, 'Min']
    b_max = bracket_row.loc[:, 'Max']
    b_con = bracket_row.loc[:, 'Constant']
    
    f1 = tn + b_con
    f2 = hs * (1 - eess - eemd)
    f3 = b_pr * (al * b_al + b_min)
    f4 = 1 - eess - eemd - b_pr   
    
    tg = ((f1 - f2 - f3) / f4)
    
#     # Basic Way
#     gross_target.value = round(((tg + hs) * (1 + erss + ermd) * (1 + atr)) / (1 - adf), 2)
#     admin_fee.value = round((((tg + hs) * (1 + erss + ermd) * (1 + atr)) / (1 - adf)) * adf, 2)
#     attrn_fee.value = round((tg + hs) * (1 + erss + ermd) * atr, 2)
#     housing.value = hs
#     er_taxes.value = round((tg + hs) * (erss + ermd), 2)
#     allowances.value = al * b_al
#     taxable_gross.value = round(tg, 2)
#     fit.value = round((tg - (al * b_al) - b_min) * b_pr + b_con, 2)
#     eess_amt.value = round((tg + hs) * eess, 2)
#     eemd_amt.value = round((tg + hs) * eemd, 2)
#     net_pay.value = round(tn - hs, 2)
#     total_net.value = tn

#     allowance_rt.value = b_al
#     allowances.value = al * b_al
#     wage_amt.value = round(tg - al * b_al, 2)
#     bracket_min.value = b_min
#     bracket_max.value = b_max
#     bracket_con.value = b_con
#     bracket_pr.value = b_pr
    
    
    # Simplified Faster way (Having to do redundant math only once.)
    b1 = tg + hs
    b2 = erss + ermd
    b3 = 1 + b2
    b4 = 1 + atr
    b5 = 1 - adf
    b6 = (b1 * b3 * b4) / b5
    b7 = al * b_al
    b8 = tg - b7
    
    gross_target.value = gross_target_slider.value = round(b6, 2)
#     gross_target_slider.value = round(b6, 2)
    
    admin_fee.value = round(b6 * adf, 2)
    attrn_fee.value = round(b1 * b3 * atr, 2)
    er_taxes.value = round(b1 * b2, 2)
    taxable_gross_housing.value = round(b1, 2)
    taxable_gross.value = round(tg, 2)
    fit.value = round((b8 - b_min) * b_pr + b_con, 2)
    eess_amt.value = round(b1 * eess, 2)
    eemd_amt.value = round(b1 * eemd, 2)
    net_pay.value = round(tn - hs, 2)
    housing.value = hs
    total_net.value = tn
    
    allowances.value = b7
    allowance_rt.value = b_al
    allowances.value = b7
    wage_amt.value = round(b8, 2)
    bracket_min.value = b_min
    bracket_max.value = b_max
    bracket_con.value = b_con
    bracket_pr.value = b_pr
    
    extra_total.value = round(b6 - tn, 2)
    extra_pr.value = round((b6 / tn - 1) * 100, 2)
    
    
    
    
# Function that runs if gross target if fixed
def run_calc_target(tax_t):
    """
    Function that runs calculations and adjusments
    when Gross Target is fixed.
    """
    
    gt = gross_target_slider.value
    tn = total_net_slider.value
    hs = housing_slider.value
    al = allowances_slider.value
    atr = attrn_fee_slider.value
    adf = admin_fee_slider.value
    
    # Get columns needed for WA calculation
    erss = tax_t.loc[:, 'eess'].unique()[0]
    ermd = tax_t.loc[:, 'eemd'].unique()[0]
    b_al = tax_t.loc[:, 'Allowance'].unique()[0]

    # Arithmetic that gets from gross target to taxable gross.
    b1 = gt * adf
    b2 = gt - b1
    b3 = (b2 * atr) / (1 + atr)
    b4 = b2 - b3 
    b5 = erss + ermd
    b6 = (b4 * b5) / (1 + b5)
    tg = b4 - b6 - hs 
    b7 = al * b_al
    b8 = tg - b7
    b9 = tg + hs
    
    admin_fee.value = round(b1, 2)
    attrn_fee.value = round(b3, 2)
    er_taxes.value = round(b6, 2)
    taxable_gross_housing.value = round(b9, 2)
    taxable_gross.value = round(tg, 2)
    allowances.value = b7
    wage_amt.value = round(b8, 2)
    
    
    # Get Wage amount column
    wage_amt_column = b8
    
    # Find row in Tax table where wage amount is inbetween min and max
    bracket_row = get_tax_table_bracket_row(tax_t, wage_amt_column)
    
    # Use bracket row to find remaining Amounts
    eess = bracket_row.loc[:, 'eess']
    eemd = bracket_row.loc[:, 'eemd']
    b_pr = bracket_row.loc[:, 'Percent']
    b_min = bracket_row.loc[:, 'Min']
    b_max = bracket_row.loc[:, 'Max']
    b_con = bracket_row.loc[:, 'Constant']

    
    c1 = (b8 - b_min) * b_pr + b_con
    c2 = b9 * eess
    c3 = b9 * eemd
    c4 = tg - c1 - c2 - c3
    c5 = tn = c4 + hs
    
    fit.value = round(c1, 2)
    eess_amt.value = round(c2, 2)
    eemd_amt.value = round(c3, 2)
    net_pay.value = round(c4, 2)
    total_net.value = round(c5 ,2) # forces change on slider now that they are linked
    
    allowance_rt.value = b_al
    allowances.value = b7
    bracket_min.value = b_min
    bracket_max.value = b_max
    bracket_con.value = b_con
    bracket_pr.value = b_pr
    
    extra_total.value = round(gt - tn, 2)
    extra_pr.value = round((gt / tn - 1) * 100, 2)

       
def obs(change):
    """
    Function that handles the changes observed when widget values change.
    """
    
    sliders_that_change_tax_table = ['Tax Year', 'Pay Freq', 'Filing']
    
    if change.owner.description in sliders_that_change_tax_table:
        global tax_table_filtered 
        tax_table_filtered = get_tax_table()
    
    global view
    if change.owner.description == 'Gross Target':
        view = 'target'
    elif change.owner.description == 'Total Net':
        view = 'net'

    # Determine fixed view, unobserve contrasting slider, 
    # run calc for fixed slider, observe contrasting slider
    if view == 'target':
        total_net_slider.unobserve(obs)
        run_calc_target(tax_table_filtered)
        total_net_slider.observe(obs)

    elif view ==  'net':
        gross_target_slider.unobserve(obs)
        run_calc_net(tax_table_filtered)
        gross_target_slider.observe(obs)
        
    # Set Housing Slider Max after observed change.
    housing_slider.max = total_net_slider.value
        
        
# Initialize tax_table_filtered
tax_table_filtered = get_tax_table()
run_calc_net(tax_table_filtered)
view = 'net'
    
# Change tax_table_filtered when changes are made to tax sliders, then run_calc
tax_year_slider.observe(obs)
pay_freq_slider.observe(obs)
filing_slider.observe(obs)

gross_target_slider.observe(obs)
admin_fee_slider.observe(obs)
attrn_fee_slider.observe(obs)
allowances_slider.observe(obs)
housing_slider.observe(obs)
total_net_slider.observe(obs)

# Create Links between sliders and Float Texts
gross_target_link = wg.link((gross_target_slider, 'value'), (gross_target, 'value'))
housing_link = wg.link((housing_slider, 'value'), (housing, 'value'))
total_net_link = wg.link((total_net_slider, 'value'), (total_net, 'value'))

# Create tandum Hboxes of FloatText and Sliders
gross_target_box = HBox([gross_target, gross_target_slider])
admin_fee_box = HBox([admin_fee, admin_fee_slider])
attrn_fee_box = HBox([attrn_fee, attrn_fee_slider])
allowance_box = HBox([allowances, allowances_slider])
housing_box = HBox([housing, housing_slider])
total_net_box = HBox([total_net, total_net_slider])

# Create layout of boxes
vbox1 = VBox([
    tax_year_slider,
    pay_freq_slider,
    filing_slider,
    bracket_min,
    wage_amt,
    bracket_max,
    allowance_rt,
    bracket_con,
    bracket_pr,
    extra_total,
    extra_pr
])

vbox2 = VBox([
    gross_target_box,
    admin_fee_box,
    attrn_fee_box,
    er_taxes,
    taxable_gross_housing,
    taxable_gross,
    allowance_box,
    fit,
    eess_amt,
    eemd_amt,
    net_pay,
    housing_box,
    total_net_box
])
all_boxes = HBox([vbox1, vbox2])

#2 For Displaying in a different Cell
output.clear_output()
with output:
        display(all_boxes)
        
# display(all_boxes)

#3 For Displaying in a different Cell
display(output)